# Chatbot Climbing Shoes 🧗


In [8]:
# imports

import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display

In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

In [3]:
# Initialize

openai = OpenAI()
MODEL = 'gpt-4o-mini'

In [48]:
system_message = "You are a helpful assistant. Expert in climbing shoes and climbing gear. You are given a description of a list of climbing shoes. Respond using markdown format"


In [5]:
# Simpler than in my video - we can easily create this function that calls OpenAI
# It's now just 1 line of code to prepare the input to OpenAI!

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
# get climbing shoes details
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}


class ClimbingShoesLandingPage:
    shoes_links = []
    base_url = "https://www.climbingzone.es/"
    url = base_url + "pies-de-gato/"

    def __init__(self):
        response = requests.get(self.url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        products_div = soup.find('div', id='js-product-list')

    
        self.shoes_links = [link.get('href') for link in products_div.find_all('a')]


landing_page = ClimbingShoesLandingPage()
landing_page.shoes_links


In [ ]:
class ClimbingShoesDetails:
    price = 'unknown'
    model = 'unknown'
    link = 'unknown'
    description = 'unknown'
    brand = 'unknown'

    def __init__(self, link):
        self.link = link
        self.response = requests.get(self.link, headers=headers)
        self.body = self.response.content
        self.soup = BeautifulSoup(self.body, 'html.parser')
        self.get_price()
        self.get_model()
        self.get_description()

    def get_price(self):
        price_div = self.soup.find('span', class_='price_with_tax price_pvp')
        if price_div:
            self.price = price_div.get_text(strip=True)

    def get_model(self):
        model_div = self.soup.find('h1', class_='product_name')
        if model_div:
            self.model = model_div.get_text(strip=True)

    def get_description(self):
        description_div = self.soup.find('div', id='description')
        if description_div:
            self.description = description_div.get_text(separator="\n", strip=True)

    def shoes_content(self):
        content = f"""
        Shoes content:\n
        Model: {self.model}\n
        Price: {self.price}\n
        Description: {self.description}\n
        Website: {self.link}\n\n
        """
        return content


example = ClimbingShoesDetails("https://www.climbingzone.es/pies-de-gato/1061-ondra-comp-la-sportiva.html")
example.shoes_content()


'\n        Shoes content:\n\n        Model: La Sportiva Ondra Comp\n\n        Price: 174,97\xa0€\n\n        Description: El\nOndra Comp es un pie de gato\npensado y desarrollado para ofrecer las mejores prestaciones en tus escaladas en rocódromo o sobre grandes desplomes en roca, ya que es un pie de gato muy blando que logra hacerte sentir hasta el más mínimo relieve en tus pies.\nCon el\nOndra Comp\ntendrás un pie de gato muy flexible para conseguir la mayor superfice de contacto y aumentar la fricción sobre volúmenes o presas grandes, sin olvidarnos que incorpora la palmilla patentada\nSenseGrip\nque le otorga una muy buena resistencia a la torsión justo bajo los dedos de los pies para las presas más pequeñas.\nAdemás de esto el\nOndra Comp\ncuenta con un engomado total en el upper para movimientos de empeine sobre volumenes, que incluso incluye una placa de protección para evitar los dolorosos dinámicos de empeine cuando no aciertas justo en el lugar donde debías. ¡Increible!\nEl On

In [47]:
climbing_shoes_details = []
for link in landing_page.shoes_links:
    climbing_shoes_details.append(ClimbingShoesDetails(link))

print(climbing_shoes_details)

[<__main__.ClimbingShoesDetails object at 0x169b02090>, <__main__.ClimbingShoesDetails object at 0x169b01f50>, <__main__.ClimbingShoesDetails object at 0x175eeca90>, <__main__.ClimbingShoesDetails object at 0x168fb1ed0>, <__main__.ClimbingShoesDetails object at 0x14fb95010>, <__main__.ClimbingShoesDetails object at 0x16906b590>, <__main__.ClimbingShoesDetails object at 0x175fd23d0>, <__main__.ClimbingShoesDetails object at 0x175eab650>, <__main__.ClimbingShoesDetails object at 0x175e72210>, <__main__.ClimbingShoesDetails object at 0x175fa5ad0>, <__main__.ClimbingShoesDetails object at 0x175d6f190>, <__main__.ClimbingShoesDetails object at 0x177ce4cd0>, <__main__.ClimbingShoesDetails object at 0x30b8122d0>, <__main__.ClimbingShoesDetails object at 0x30ccea510>, <__main__.ClimbingShoesDetails object at 0x30cedf890>, <__main__.ClimbingShoesDetails object at 0x30d1662d0>, <__main__.ClimbingShoesDetails object at 0x30d3c7e10>, <__main__.ClimbingShoesDetails object at 0x30d845710>, <__main__

In [49]:
system_message = "You are a helpful assistant. Expert in climbing shoes and climbing gear.  Respond using markdown format. You are given a description of a list of climbing shoes with the following format: \n\n Shoes content:\n Model: {self.model}\n Price: {self.price}\n Description: {self.description}\n Website: {self.link}\n\n"

for detail in climbing_shoes_details:
    system_message += f"\n\n{detail.shoes_content()}"

print(system_message)


You are a helpful assistant. Expert in climbing shoes and climbing gear.  Respond using markdown format. You are given a description of a list of climbing shoes with the following format: 

 Shoes content:
 Model: {self.model}
 Price: {self.price}
 Description: {self.description}
 Website: {self.link}




        Shoes content:

        Model: Garra Kamae

        Price: 61,50 €

        Description: Modelo fabricado en piel natural, con suela de goma cocida de 4,5mm y 2 cierres de velcro que ofrece una comodidad excepcional.
El Kamae permite introducirse en la escalada con muy buenas prestaciones y acabados. Es un pie de gato que te durará mucho tiempo

        Website: https://www.climbingzone.es/pies-de-gato/558-kamae-garra.html


        


        Shoes content:

        Model: La Sportiva Ondra Comp

        Price: 174,97 €

        Description: El
Ondra Comp es un pie de gato
pensado y desarrollado para ofrecer las mejores prestaciones en tus escaladas en rocódromo o sobre grand

In [50]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [51]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7875

To create a public link, set `share=True` in `launch()`.
